<a href="https://colab.research.google.com/github/tavi1402/Pytorch/blob/main/linera_regression__with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.0+cpu (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2)
ERROR: No matching distribution found for torch==1.7.0+cpu


## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

In [1]:
import numpy as np
import torch

### **Step1** Take the input as an array

In [2]:
# input= teprature, humidity,rainfall
inputs= np.array([[73,67,43],
                [91,88,64],
                [87,134,58],
                [102,43,37],
                [69,96,70]], dtype="float32")

In [3]:
#targets=[apple and oranges ]
targets= np.array([[56,70],
                  [81, 101],
                  [119,133],
                  [22,37],
                  [103,119]], dtype= "float32")

###  **Step2**- Convert input and targets into tensors

In [4]:
inputs = torch.from_numpy(inputs)
targets= torch.from_numpy(targets)

In [5]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [6]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### **Step-3** Initailise the weight and biases

In [6]:
w= torch.randn(2,3, requires_grad=True)
b= torch.randn(2, requires_grad= True)

In [7]:
print(w, b)

tensor([[ 0.6482, -1.3448, -0.0522],
        [-0.1108,  0.4250, -2.2583]], requires_grad=True) tensor([-1.3681,  1.1727], requires_grad=True)


In [8]:
# w*x+b
def model(x):
  return x @ w.t() + b


In [9]:
preds= model(inputs)
preds

tensor([[ -46.3980,  -75.5520],
        [ -64.0686, -116.0473],
        [-128.2109,  -82.5033],
        [   4.9904,  -75.4167],
        [ -89.4019, -123.7587]], grad_fn=<AddBackward0>)

In [10]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### **Step4** Calculate the error

In [11]:
def mse(y1,y2):
  diff=y1-y2
  return torch.sum(diff*diff)/ diff.numel()

In [12]:
loss=mse(preds,targets)
loss

tensor(31625.7188, grad_fn=<DivBackward0>)

### Now to optimise the loss apply **backword propogation** to update the weight and bias.


In [13]:
loss.backward()

In [14]:
print(w) # thisn is the same weights before backword propogation

tensor([[ 0.6482, -1.3448, -0.0522],
        [-0.1108,  0.4250, -2.2583]], requires_grad=True)


In [15]:
print(w.grad) # this is the upgraded weight after backword propgation

tensor([[-11438.8721, -14390.9922,  -8424.6455],
        [-15468.4492, -17173.6680, -10760.2959]])


In [16]:
print(b.grad)

tensor([-140.8178, -186.6556])


In [17]:
with torch.no_grad():
  w -=w.grad * 1e-5    # we are multiplying our wights with the very small(learning rate)
  b -= b.grad * 1e-5

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [19]:
pred= model(inputs)
print(preds)

tensor([[ -46.3980,  -75.5520],
        [ -64.0686, -116.0473],
        [-128.2109,  -82.5033],
        [   4.9904,  -75.4167],
        [ -89.4019, -123.7587]], grad_fn=<AddBackward0>)


In [20]:
loss=mse(preds,targets)
loss

tensor(31625.7188, grad_fn=<DivBackward0>)

In [22]:
# # Compute gradients
# loss.backward()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([-140.8178, -186.6556])


In [23]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [24]:
preds=model(inputs)
preds

tensor([[-24.7803, -48.1231],
        [-35.6006, -79.9678],
        [-94.0860, -39.7883],
        [ 25.9661, -48.2692],
        [-61.7937, -89.0628]], grad_fn=<AddBackward0>)

In [25]:
loss = mse(preds, targets)
print(loss)

tensor(21981.8691, grad_fn=<DivBackward0>)


In [26]:
for i in range(100):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b-=b.grad*1e-5

    w.grad.zero_()
    b.grad.zero_()

In [27]:
pred= model(inputs)
loss=mse(preds,targets)
loss

tensor(795.4769, grad_fn=<DivBackward0>)

In [28]:
preds

tensor([[ 66.5071,  75.2219],
        [ 88.9296,  85.3054],
        [ 88.3806, 160.0083],
        [ 76.7460,  62.5358],
        [ 80.8630,  77.7489]], grad_fn=<AddBackward0>)

In [29]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [30]:
for i in range(100):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b-=b.grad*1e-5

    w.grad.zero_()
    b.grad.zero_()

In [31]:
pred= model(inputs)
loss=mse(preds,targets)
loss

tensor(388.4275, grad_fn=<DivBackward0>)

In [32]:
preds

tensor([[ 62.0400,  73.7898],
        [ 85.5493,  85.8819],
        [103.2500, 160.9996],
        [ 50.7492,  54.3675],
        [ 90.1525,  83.5185]], grad_fn=<AddBackward0>)

In [33]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [34]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')


In [35]:
inputs=torch.from_numpy(inputs)
targets= torch.from_numpy(targets)

# TensorDataset(inputs, targets):

- It's like putting your data into two separate boxes (tensors).
- The first box (inputs) contains the information you want the computer to learn from (like images or numbers).
- The second box (targets) contains the correct answers or labels that you want the computer to learn to predict

In [36]:
from torch.utils.data import TensorDataset

In [37]:
train_df=TensorDataset(inputs, targets)
train_df[:5]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [38]:
from torch.utils.data import DataLoader


In [39]:
batch_size=5
train_dataloader=DataLoader(train_df,batch_size, shuffle=True)
train_dataloader

In [40]:
for inputs, targets in train_dataloader:
  print(inputs)
  print(targets)


tensor([[ 73.,  67.,  43.],
        [ 74.,  66.,  43.],
        [102.,  43.,  37.],
        [101.,  44.,  37.],
        [ 88., 134.,  59.]])
tensor([[ 56.,  70.],
        [ 57.,  69.],
        [ 22.,  37.],
        [ 21.,  38.],
        [118., 132.]])
tensor([[ 68.,  96.,  71.],
        [ 91.,  88.,  64.],
        [ 73.,  66.,  44.],
        [ 91.,  87.,  65.],
        [ 87., 134.,  58.]])
tensor([[104., 118.],
        [ 81., 101.],
        [ 57.,  69.],
        [ 80., 102.],
        [119., 133.]])
tensor([[ 68.,  97.,  70.],
        [ 69.,  96.,  70.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.]])
tensor([[102., 120.],
        [103., 119.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.]])


In [41]:
import torch
import torch.nn as nn

In [48]:
model= nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1296,  0.1044,  0.2387],
        [ 0.2203,  0.3832, -0.5658]], requires_grad=True)
Parameter containing:
tensor([-0.3064,  0.1922], requires_grad=True)


In [49]:
#another way to find to weight and bias
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1296,  0.1044,  0.2387],
         [ 0.2203,  0.3832, -0.5658]], requires_grad=True),
 Parameter containing:
 tensor([-0.3064,  0.1922], requires_grad=True)]

In [50]:
preds=model(inputs)
preds

tensor([[26.4147, 17.6182],
        [35.9533, 17.7484],
        [38.8058, 37.8899],
        [26.2343, 18.2050],
        [35.3702, 12.5724],
        [26.4399, 17.4553],
        [36.0877, 16.7993],
        [39.1741, 37.5444],
        [26.2091, 18.3679],
        [35.4793, 11.7862],
        [26.5490, 16.6691],
        [35.9785, 17.5855],
        [38.6715, 38.8390],
        [26.1251, 18.9911],
        [35.3450, 12.7353]], grad_fn=<AddmmBackward0>)

In [51]:
import torch.nn.functional as F

In [52]:
loss_fn=F.mse_loss
loss_fn

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Optional[bool] = None, reduce: Optional[bool] = None, reduction: str = 'mean') -> torch.Tensor>

In [53]:
loss = loss_fn(preds, targets)
print(loss)

tensor(4430.0889, grad_fn=<MseLossBackward0>)


In [59]:
loss.item()

4430.0888671875

In [55]:
opt= torch.optim.SGD(model.parameters(), lr= 1e-5)

In [68]:
def fit(num_epochs,model,  loss_fn, opt, train_dataloader):
  for epoch in range(num_epochs):

    for input, target in train_dataloader:
      preds= model(inputs)

      loss= loss_fn(preds,targets)

      loss.backward()

      opt.step()

      opt.zero_grad()

    if (epoch+1)%10==0:
      print(f'Epocs {epoch+1}/{num_epochs}, loss: loss.items{loss.item():.4f}') # get teh numerical value of the loss function



In [69]:
fit(100,model, loss_fn, opt, train_dataloader)

Epocs 10/100, loss: loss.items51.0466
Epocs 20/100, loss: loss.items46.4624
Epocs 30/100, loss: loss.items42.6878
Epocs 40/100, loss: loss.items39.5019
Epocs 50/100, loss: loss.items36.7522
Epocs 60/100, loss: loss.items34.3329
Epocs 70/100, loss: loss.items32.1698
Epocs 80/100, loss: loss.items30.2110
Epocs 90/100, loss: loss.items28.4191
Epocs 100/100, loss: loss.items26.7673


In [70]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.1741,  71.1517],
        [ 81.1975,  95.8668],
        [119.8089, 142.5872],
        [ 22.2554,  41.1364],
        [ 99.5028, 108.4106],
        [ 55.9352,  69.9891],
        [ 80.8823,  95.0772],
        [120.0196, 142.7092],
        [ 23.4943,  42.2990],
        [100.4265, 108.7837],
        [ 56.8589,  70.3622],
        [ 79.9586,  94.7042],
        [120.1240, 143.3767],
        [ 21.3317,  40.7633],
        [100.7417, 109.5732]], grad_fn=<AddmmBackward0>)

In [71]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [72]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4988, 67.0000]], grad_fn=<AddmmBackward0>)

In [73]:
model(torch.tensor([79,89,77.]))

tensor([ 93.7315, 101.6283], grad_fn=<AddBackward0>)